In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer,LancasterStemmer
import re

In [2]:
def pred(y): 
    '''
    This function maps the probabilities outputed by the model back to the rankings list 
    and outputs the ranking with the highest probability. 
    
    inputs: 
    y  (1,m)     : Probability output of the RNN model 
    
    outputs: 
    res (string) : The ranking corresponding to the most probable outcome. 
    
    '''
    y = y.tolist()
    #ranking = ['Below Average' , 'Average' , 'Above Average']
    res = ranking[y.index(max(y))]
    return(res)




In [3]:
#need to write another function that maps the correct output of the function to the rankings. 
def vec_output(y): 
    m = len(ranking)
    txt = y
    v = np.zeros(m) 
    j = ranking.index(txt)
    v[j] = 1
    return v 


In [4]:
def accuracy(y, y_actual):
    
    accuracy = 0
    right_pred = np.zeros(len(y))
    for i in range(0,len(y)): 
        right_pred[i] = pred(y[i]) == y_actual[i]
    
    accuracy = sum(right_pred)/len(y) * 100
    accuracy = format(accuracy, ".2f")  
    #print(f"The accuracy of the model is {accuracy}%")
    return(accuracy)
    
    

In [5]:
#Loading the data: 
CustomerFeed = 'Canva_reviews.xlsx'
df = pd.read_excel(CustomerFeed)

print(df)

                                               reviewId            userName  \
0     gp:AOqpTOFxf3fttcT5DSvFIn9KPp5FErgH9yC533Fmoxv...      Donna Caritero   
1     gp:AOqpTOEq6rNIWLnPV4KFTctWvm0mpGEQljtD6mvy1H-...  Soumi Mukhopadhyay   
2     gp:AOqpTOE86hSyPRHZgYt28Uk5zGe4FZGb1hkmtFDiYJ2...   Theknown _unknown   
3     gp:AOqpTOHSuKkVTcM3QgCCKysHQlxEnk2ocOKsUMiMIJy...        Anthony Dean   
4     gp:AOqpTOEOrZt5H6jXPiplJyffCd5ZBnVXACTWgwNsF1R...   Neha Diana Wesley   
...                                                 ...                 ...   
1495  gp:AOqpTOHhnXMpylU3f-1V1KbR2hwWArOilxPlKI6K4xY...            Reen Ali   
1496  gp:AOqpTOEcz62DHS-amqTB5xGMhM4_R0UJpcv_HDNny9i...     Shaurya Chilwal   
1497  gp:AOqpTOFMqEqa_kpp29Q8wjcBmKUCAvOQGQx4KZQ8b83...           GK Gaming   
1498  gp:AOqpTOGY4z3pUxeiqGzn2ad3Noxqlbm-9DZ3ksHqD1_...    1203_Vani Sharma   
1499  gp:AOqpTOFVGZ0MXyR-Gv_d2cYf2KD709Hwple_u7OZE4y...           MeLLy EcK   

                                              userI

In [6]:
df = df[["review", "Sentiment"]]
df.head()

,review,Sentiment
0,Overall it's really an amazing app. I've been ...,Negative
1,Hey! Yes I gave a 5 star rating... coz I belie...,Positive
2,Canva used to be a good app! But recently I've...,Negative
3,"It's a brilliant app, but I have just one prob...",Negative
4,This was such a great app. I used to make BTS ...,Negative


In [7]:
reviews = df["review"] 
reviews.head()
reviews.shape

(1500,)

Let's work on the reviews to simplify the input: 

In [8]:
def edit_txt(review):
    """
    This function receives a text and returns it edited as follows: 
    1, all words converted to lower case 
    2, integers removed
    3, tokenize the words 
    4, punctuation removed 
    5, common words that are unnecessary are removed. 
    """
    
    review_edited = []

    #Converting to lower case: 
    review_edited = review.lower() 
    
    #Removing integers: 
    pattern = r'[0-9]'
    # Match all digits in the string and replace them with an empty string
    review_edited = re.sub(pattern, '', review_edited) 

    #Tokenize the comment: 
    review_edited = word_tokenize(review_edited) 

    #Removing punctuation 
    tokenizer = RegexpTokenizer(r'\w+')
    review_edited = [''.join(tokenizer.tokenize(word)) for word in review_edited if len(tokenizer.tokenize(word))>0]

    #Removing common words: 
    remove_list = stopwords.words('english') 
    to_remove = [ "not",'don',"don't",'should',"should've", 'ain','aren',"aren't",'couldn',"couldn't",'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn', "wouldn't"]
 
    review_edited = [word for word in review_edited if not word in remove_list]
    return(review_edited) 

   


In [9]:
reviews_edited = [edit_txt(review) for review in reviews]
print(reviews_edited[13])
print(reviews[13])

['unable', 'save', 'work', 'nothing', 'works']
Unable to save my work. Nothing works :(


In [10]:
#5 optional:find the stem of every word so that words like working, work, and worked are inputted equivantly in the model 
stem_token = PorterStemmer()
print(stem_token.stem("working"))

reviews_edited = [[stem_token.stem(word) for word in review] for review in reviews_edited]
reviews_edited[13]

work


['unabl', 'save', 'work', 'noth', 'work']

In [11]:
lancaster = LancasterStemmer()
reviews_edited = [[lancaster.stem(word) for word in review] for review in reviews_edited]
reviews_edited[13]

['un', 'sav', 'work', 'noth', 'work']

In [12]:
dffed = df.iloc[:,0]
x = reviews
dfrank = df.iloc[:,1]

#print(x[10])
print(reviews[11])
print(reviews_edited[11])
type(x)

having troubles downloading the files recently, even after cleaning up the cache. it just stuck at the downloading page ("your so called making magic happens window"). I even went to the extend of uninstall and re-install the app and still the same. just recently this problem occurs, before this it was just alright. for a premium user, if this problem still hasn't been improved, it's disappointing. might as well just discontinue the membership.
['troubl', 'download', 'fil', 'rec', 'ev', 'cle', 'cach', 'stuck', 'download', 'pag', 'cal', 'mak', 'mag', 'hap', 'window', 'ev', 'went', 'extend', 'uninst', 'reinst', 'ap', 'stil', 'rec', 'problem', 'occ', 'alright', 'prem', 'us', 'problem', 'stil', 'nt', 'improv', 'disappoint', 'might', 'wel', 'discontinu', 'memb']


pandas.core.series.Series

In [13]:
y = df["Sentiment"].tolist()
ranking = np.unique(y)
ranking = ranking.tolist()
ranking

['Negative', 'Positive']

In [14]:
#creating the dictionary: 
lenx = len(x)

Split = [] 
Dic = []
dictionary = np.unique([word for review in reviews_edited for word in review]).tolist()
dictionary[1:10]


['aa', 'aap', 'abl', 'absolv', 'acc', 'acceiv', 'access', 'accid', 'accod']

In [15]:
#Let us try a different word embedding; this time we will use the glove methond and an online dictionary: 
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector
        

In [16]:
words =  list(embeddings_dict.keys())
vectors = [embeddings_dict[word] for word in words]

In [17]:
#dividing the dataset into 75% training set and 25% test set: 
x = x.to_list()
X_train, X_test, y_train, y_test = train_test_split(x,y, 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True) 


In [18]:
type(y)

list

In [19]:
X_train = [edit_txt(comment) for comment in X_train]
X_test = [edit_txt(comment) for comment in X_test]

In [20]:
type(X_train)

list

**Update**

- a global dictionary is used instead of our own dictionary

       -> new customer feedbacks can also be evaluated. 
       -> the new word embedding in place can recognize word similarities 
       -> avoid large sparse one-hot vectors for the sake of memory usage.

In [21]:
print(words[0:40])
len(words)

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s", 'for', '-', 'that', 'on', 'is', 'was', 'said', 'with', 'he', 'as', 'it', 'by', 'at', '(', ')', 'from', 'his', "''", '``', 'an', 'be', 'has', 'are', 'have', 'but', 'were', 'not', 'this', 'who', 'they']


400000

In [22]:
words[0]

'the'

In [23]:
vectors[0]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [24]:
embeddings_dict["."].shape

(50,)

In [25]:
#we want to redefine the function so that if it has multiple sentences, it will run the below but it there's only one, it will 
#take an alternative approach. 

def gvec_input(x): 
    """
    This function takes any input, x, and returns a glove vector based on the 
    words introduced in the vocabulary (400,000 words). This function returns k vectors where k is the number of words in the 
    sentence. Every vector corresponds to a word in the dictionary and each entry will describe a feature of the word. 
    
    inputs: 
    
    x (string) : a statement from customers. 
    
    outputs: 
    v (m,n)    : where m is the number of words in the sentence and n = 50 is the number of total features describing a word. 

    
    """
    n = len(x)
    m = 50 #size of the standard input: 
    gv = np.zeros((n,m, 50))
    for i in range(0, n): #looping over each comment 
        txt = x[i]
        g = len(txt)
        for l in range(g): #looping over each word 
            #if a word is not is the list of Glove embeddings, then assign an array which is the average of all embeddings: 
            if txt[l] not in words: 
                gv[i,l,:] = np.mean(vectors, axis = 0)
            else: 
                gv[i,l,:] = embeddings_dict[txt[l]]
        for h in range(g,m): 
            gv[i,h,:] = np.ones(50)
    return(gv)

In [26]:
#converting x_train and x_test to word embeddings: 
X_trainmod = gvec_input(X_train)
X_testmod = gvec_input(X_test) 

In [27]:
def Softmax(N): 
    N = N.astype(np.float64)
    N = N.tolist()
    n = len(N) 
    k = len(N[0]) 
    res = np.zeros((n,k)) 
    #looping over each sample: 
    for i in range(0,n): 
        #N[i] = N[i] - np.max[N[i]]
        Sum = sum(np.exp(N[i])) 
        for j in range(k): 
            res[i,j] = np.exp(N[i][j])/Sum 
    return(res) 


**LSTM Model**

Note : also apply to the time series dataset. then apply the tenserflow version and see if there is any difference with the simple RNN model. Maybe the simple RNN was just not able to pick up the patterns (which is not feasible since the RNN made by us were working at least with 60% accuracy) .

In [28]:
#initialize parameters: 
parameters = {} 

#dimension of every hidden neuron  
n = vectors[0].shape[0] #size of embeddings 
m = 10                  #every hidden neuron has the same dim as the input (1*m) m = 6
b = len(ranking)        #categories of the output 
z = 10                  #number of units for the cell state 
g = X_trainmod.shape[0] #number of samples 
#Initialize weights: 
parameters["forget"] = np.random.randn(m + n, z)
parameters["update"] = np.random.randn(m + n, z)
parameters["ihidden"] = np.random.randn(m + n, z)
parameters["candid"] = np.random.randn(m + n, z)
parameters["output_weights"] = np.random.randn(m,b)

#Initializing biases: 
parameters["b_f"] = np.random.randn(z)
parameters["b_u"] = np.random.randn(z)
parameters["b_i"] = np.random.randn(z) #?
parameters["b_c"] = np.random.randn(z)
parameters["b_y"] = np.random.randn(b)

#initial hidden and cell states 
parameters["a_prev"] = np.tile(np.random.randn(1,m), (g,1)).reshape(g,1,m) #this make a copy of a_prev so we initiate all hidden states for all samples at the same time 
parameters["cell_prev"] = np.tile(np.random.randn(1,z),(g,1)).reshape(g,1,z)

cache = {}


In [29]:
def lstm_cell(xt, parameters):
    """
   
    """
    sample_size = xt.shape[0]
    
    #Extract parameters: 
    w_f = parameters["forget"]
    w_u = parameters["update"]
    w_i = parameters["ihidden"]
    w_c = parameters["candid"]
    w_y = parameters["output_weights"]
    
    b_f = parameters["b_f"]
    b_u = parameters["b_u"]
    b_i = parameters["b_i"]
    b_c = parameters["b_c"]
    b_y = parameters["b_y"]

    a_prev = parameters["a_prev"][:,-1,:].reshape(sample_size, 1, 10)
    cell_prev = parameters["cell_prev"][:,-1,:].reshape(sample_size, 1, 10)

    cache = ()
    #concatenate the input and the hidden layer: 
    conc_res = np.concatenate((xt,a_prev.reshape(sample_size,a_prev.shape[2])), axis = 1)

    #Calculating the gates and save into the cache: 

    #forget gate: 
    forget = Softmax(np.dot(conc_res,w_f) + b_f)
    #update gate: 
    update = Softmax(np.dot(conc_res,w_u) + b_u)
    #candidate cell state: 
    candid = np.tanh(np.dot(conc_res, w_c) + b_c)
    #i gate: 
    ihidden = Softmax(np.dot(conc_res, w_i) + b_i)
    
    #Updating the new cell state and parameter cell_next for next step: 
    cell_next = (forget * cell_prev[:,-1,:] + update * candid).reshape(sample_size, 1, cell_prev.shape[2])
    parameters["cell_prev"]= np.concatenate((parameters["cell_prev"],cell_next), axis = 1)
    
    #Updating the new hidden state and parameter a_prev for next step: 
    a_next = (np.tanh(cell_next[:,-1,:]) * ihidden).reshape(sample_size, 1, a_prev.shape[2])
    parameters["a_prev"] = np.concatenate((parameters["a_prev"],a_next), axis = 1)
    
    #saving all the parameters: 
    #cache = {"forget" : forget, "update" : update, "candid" : candid, "ihidden" : ihidden}
    cache = (forget, update, candid, ihidden)
    
    return(a_next,cell_next, cache, parameters)



In [30]:
a_next,cell_next, cache, parameters = lstm_cell(X_trainmod[:,0,:], parameters)


In [31]:
parameters["a_prev"].shape

(1125, 2, 10)

In [32]:
cache[0].shape #this is the forget gate calculated for all 1125 training samples 

(1125, 10)

In [33]:
cache[1][0] #this is the update gate for the first sample in the training 

array([5.15756682e-04, 3.65098893e-07, 1.80765761e-01, 4.85120299e-05,
       8.04847614e-01, 2.48871153e-03, 3.52073548e-06, 7.95372747e-07,
       1.13287986e-02, 1.65400580e-07])

In [34]:
#extract the first candidate cell_state: 
cache[2][0]

array([ 0.99999986, -0.99739466,  0.90833275,  0.99999972,  1.        ,
       -0.99999982,  0.83290237, -0.99825465, -0.99999767, -0.80048775])

In [35]:
cell_next.shape #this is the cell_state for the second element in the sequence has 10 neurons in it. all 1125 samples are included. 

(1125, 1, 10)

In [36]:
cache[3].shape #this is the i gate output for updating the hidden state at each time point. has 10 enteries corresponding to the 10 neurons of 
#the hidden state and all samples are included. now the problem is that for each step you change the parameters individually? shouldn't you add the 
#change required for each one of them and update the parameters simultanously for all of them? yes. that's right otherwise the parameters are 
#updated individually and the machine never learns from the collection of all samples. 

(1125, 10)

In [37]:
a_next.shape

(1125, 1, 10)

So far, we have designed the lstm_cell that performs a single step in LSTM forward calculations for time t. The next  step would be to figure out what parameters we want to be outputed by the lstm_step and then run this for all time steps and finally make a precition based on a_next. 

**also remember to go back at the rnn back propagation step_wise you would need to add a_next to the calculations which you've missed before.**
if we need to back propagate through time remember to sum all the change required for forget gate together after the calculation of gradients. 

figure out why b is summed over all elements for the gradient. 

could it possibly be problematic if we have too much of padding for the end of the sentences? because imagine the result of every gate would be the same and then would it make a difference of would it just confuse the machine further and make it less attentive to the beginning of the sequence? 


In [38]:
def lstm_forward(xt, parameters): 
    
    """
    this step will repeat the procedure of lstm_cell for all time steps 
    note that in order to avoid miscalculations, do not run the rnn stepward before the lstm_forward. 
    this is because the parameters are being updated while running the lstm stepwise and then starting the lstm forward just appends 
    to the resutls instead of starting over. 

    within this function the lstm forward cell will be run for the length of sequences xt. cell state and hidden state are initialized 
    randomly. In each iteration, the cell state and hidden state will be calculated; these values will be appended to parameters and outputed 
    after each lstm step. 

    a_next and cell_next will also be outputed so that the prediction can be easily made based on the last hidden state. 

    The cache will keep track of every gate output and the candidate cell state and then all of the outputs will be appended to the caches to 
    keep track of all outputs. 
    
    """
    
    n = xt.shape[1] #length of each comment 
    w_y = parameters["output_weights"]
    b_y = parameters["b_y"]
    m = w_y.shape[1]
    j = xt.shape[0]
    
    
    caches= [] #for keeping track of all caches 
    
    for i in range(n): 
        
        
        if i == n-1: #making prediction on the last step
            print(f"end of process")
            a_next, cell_next, cache, parameters = lstm_cell(xt[:,i,:], parameters)
            caches.append(cache)
            y_pred = Softmax((np.dot(a_next, w_y) + b_y).reshape(j,m))
            
        else:    
            a_next, cell_next, cache, parameters = lstm_cell(xt[:,i,:], parameters)
            caches.append(cache)

    
    return(y_pred, caches, parameters) 


In [39]:
y_pred, caches, parameters = lstm_forward(X_trainmod,parameters)

end of process


In [40]:
type(caches)

list

In [41]:
caches[0]#this is all the output of gates and cadidate gate for the first word in the sequence. there are 50 caches appended to this var.
#the parameters are forget, update, candid, ihidden respectively. how many neurons in each forget gate? must be 10 

(array([[2.15437574e-08, 4.05903329e-03, 1.57028598e-04, ...,
         1.25474824e-05, 3.97849926e-01, 1.63915839e-06],
        [5.73178039e-08, 1.41823358e-02, 4.48376642e-08, ...,
         1.93183237e-08, 3.74687926e-02, 9.08275216e-07],
        [3.87949986e-05, 3.90007152e-01, 2.28109524e-06, ...,
         2.17214737e-09, 6.07115436e-01, 6.98439213e-04],
        ...,
        [3.30460352e-09, 5.94545360e-03, 4.11256752e-10, ...,
         3.70156483e-09, 7.15066544e-01, 3.67323766e-07],
        [3.30460352e-09, 5.94545360e-03, 4.11256752e-10, ...,
         3.70156483e-09, 7.15066544e-01, 3.67323766e-07],
        [2.71721088e-07, 6.90130285e-01, 3.96879873e-08, ...,
         1.28599502e-07, 1.09097046e-01, 1.14652795e-07]]),
 array([[6.05034879e-05, 1.52794506e-06, 1.37058405e-02, ...,
         1.47766648e-04, 9.99416966e-02, 1.48149391e-03],
        [1.51158298e-05, 7.91720469e-06, 3.14743653e-01, ...,
         2.12769750e-02, 1.90750979e-04, 8.77638263e-04],
        [1.66622017e-02, 

In [42]:
caches[0][0].shape #10 neurons in the forget gate. similarly, you can check the rest. 

(1125, 10)

In [43]:
y_pred

array([[0.50939109, 0.49060891],
       [0.50938971, 0.49061029],
       [0.50939092, 0.49060908],
       ...,
       [0.50939115, 0.49060885],
       [0.50938962, 0.49061038],
       [0.50939114, 0.49060886]])

two problems: one is that the lstm might actually have a problem such that all the outputs are predicted to be the same. 


The problem is that the a_next and cell_state are dimineshed in value significantly due to the Softmax activation functions used. 
therefore, when we arrive at the last step of the sequences, a_next multiplied by w_y will cause its value to approach to zero. the Softmax is then applied only to the b_y which is why the output values are the same. 

how about normalizing the values? in softmax activation function? 


In [44]:
parameters["cell_prev"][:,-1,:] #values of the last cell state 

array([[-7.32939460e-01, -3.69897842e-07,  1.29179079e-05, ...,
        -4.37767020e-07,  9.10756521e-05,  6.74996842e-09],
       [-7.32939725e-01,  2.96008992e-05,  1.29176573e-05, ...,
        -4.37760185e-07,  9.10737995e-05,  6.74995992e-09],
       [-7.32939492e-01,  3.26812573e-06,  1.29178774e-05, ...,
        -4.37766190e-07,  9.10754272e-05,  6.74996738e-09],
       ...,
       [-7.32939448e-01, -1.69916898e-06,  1.29179190e-05, ...,
        -4.37767323e-07,  9.10757342e-05,  6.74996879e-09],
       [-7.32939741e-01,  3.14964969e-05,  1.29176415e-05, ...,
        -4.37759753e-07,  9.10736824e-05,  6.74995938e-09],
       [-7.32939449e-01, -1.52075582e-06,  1.29179175e-05, ...,
        -4.37767282e-07,  9.10757232e-05,  6.74996874e-09]])

In [45]:
parameters["a_prev"][:,-1,:]

array([[-3.48269440e-05, -1.03846391e-07,  4.05658195e-07, ...,
        -1.04809835e-09,  5.26497050e-05,  5.31124396e-10],
       [-3.48271917e-05,  8.31011921e-06,  4.05654785e-07, ...,
        -1.04808228e-09,  5.26490262e-05,  5.31124099e-10],
       [-3.48269740e-05,  9.17502987e-07,  4.05657781e-07, ...,
        -1.04809640e-09,  5.26496226e-05,  5.31124360e-10],
       ...,
       [-3.48269330e-05, -4.77030892e-07,  4.05658347e-07, ...,
        -1.04809906e-09,  5.26497351e-05,  5.31124409e-10],
       [-3.48272074e-05,  8.84227759e-06,  4.05654569e-07, ...,
        -1.04808126e-09,  5.26489833e-05,  5.31124080e-10],
       [-3.48269344e-05, -4.26942489e-07,  4.05658326e-07, ...,
        -1.04809897e-09,  5.26497310e-05,  5.31124407e-10]])

In [46]:
#reminder to correct the back propagation of the last programmed rnn_forward code. 

In [47]:
y_trainmod = np.zeros((1125,2))
for i in range(len(y_train)): 
    y_trainmod[i, :] = vec_output(y_train[i])
y_trainmod

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [48]:
parameters["cell_prev"].shape

(1125, 52, 10)

In [49]:
ranking

['Negative', 'Positive']

In [50]:
def backprop_lstm(X_trainmod, parameters, y_train, iterations, alpha): 
    """
    here we will run one step of the back propagation process. 

    note that cell_prev has 51 columns including the initialized cell state. 
    """
    #first make a prediction based on the past? or maybe what it can receive is all the parameters after the lstm_forward step? coz it already 
    #has it all? so what should we do? we take all the parameters as input. in this step you should already have run the lstm forward. 
    #then we do what? take the lstm forward outputs and unwrap cache for gates outputs and parameters for the weights, biases, and the a_prev 
    #and cell_prev. now in this case, you can run a reversed i iteration so that if i == last iteration, then we take the difference between the 
    #prediction and the y_actual. then we move thrgouh the iterations back propagating this difference to update all parameters to reflect this change. 
    
    n, T_x ,emb = X_trainmod.shape
    p_y = 2                   #number of categories (negative vs positive ->2) 
    
    #define the y_actual: 
    y_trainmod = np.zeros((n,p_y))
    
    for i in range(n): 
        
        y_trainmod[i, :] = vec_output(y_train[i])

    #unpack the parameters: 
    w_f = parameters["forget"]
    w_u = parameters["update"]
    w_i = parameters["ihidden"]
    w_c = parameters["candid"]
    w_y = parameters["output_weights"]
    
    b_f = parameters["b_f"]
    b_u = parameters["b_u"]
    b_i = parameters["b_i"]
    b_c = parameters["b_c"]
    b_y = parameters["b_y"]

   

    #store the number of hidden neurons: 
    n_a = w_y.shape[0]

    #start the iterations: 
    for j in range(iterations):
        
        #make a prediction: 
        y_pred, caches, parameters = lstm_forward(X_trainmod, parameters)#this line can be problematic if the lstm_forward is previously run. 

        #update the hidden and cell states: 
        a_state = parameters["a_prev"]
        cell_state = parameters["cell_prev"]
        
        #calculate the loss function: 
        cost = y_trainmod - y_pred
        
        dht = np.dot(cost, w_y.T) 
        dwy = np.dot(dht.T,cost)
        dby = np.sum(cost, axis = 1, keepdims = True)
        dc_next,dh_next = 0,0 
        dwf, dwu, dwihidden, dwcandid = 0,0,0,0
        dbcandid, dbf, dbu, dbihidden = 0,0,0,0
        
        for i in reversed(range(T_x)): 
            #load your cache: 
            ft, ut, candidt, ihiddent = caches[i] 
            #we take the 50th element of cell state (which is actually the last one) 
            dht = dht + dh_next 
            dct = dht * ihiddent * (1 - (np.tanh(cell_state[:,i+1,:]))**2) + dc_next 
            dcandidt = dct * ut * (1-candidt**2)

            #gate derivatives: 
            dihiddent = dht * np.tanh(cell_state[:,i+1, :]) * ihiddent * (1-ihiddent)
            dft = dct * cell_state[:,i, :] * ft * (1-ft)
            dut = dct * candidt * ut * (1-ut)
        
            #update the weights: 
            dwf += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dft) #change dims to vars. 
            dwu += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dut)
            dwihidden += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dihiddent)
            dwcandid += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dcandidt)

            dbcandid += np.sum(dcandidt, axis=1, keepdims=True) #add all the rows together. (1125 by 3 collapsed to 1 by 3) 
            dbf += np.sum(dft, axis=1, keepdims=True) 
            dbu += np.sum(dut, axis=1, keepdims=True) 
            
            dbihidden += np.sum(dihiddent, axis=1, keepdims=True) 
            dh_next = np.dot(dft, w_f[-10:, :].T) + np.dot(dcandidt, w_c[-10:, :].T) + np.dot(dihiddent, w_i[-10:, :].T) + np.dot(dut, w_u[-10:, :].T)
            dc_next = dct * ft 

        #update the parameters: 
        w_f = w_f - alpha * dwf
        w_u = w_u -alpha * dwu
        w_i = w_i - alpha * dwihidden
        w_c = w_c - alpha * dwcandid 
        w_y = w_y -alpha * dwy 
        
        b_f = b_f - alpha * dbf 
        b_u = b_u - alpha * dbu 
        b_c = b_c - alpha * dbcandid
        b_i = b_i - alpha * dbihidden
        b_y = b_y - alpha * dby 

        #Re-initialize the variables a_prev and cell_prev for the next run: 
        a_prev = np.tile(np.random.randn(1,n_a), (n,1)).reshape(n,1,n_a) #this make a copy of a_prev so we initiate all hidden states for all samples at the same time 
        cell_prev = np.tile(np.random.randn(1,n_a),(n,1)).reshape(n,1,n_a)

        #Delete the content of the caches: 
        caches = []

        #Save the updated parameters: 
        parameters = {
            "forget": w_f, "update": w_u, "ihidden": w_i, "candid": w_c, "output_weights": w_y, "b_f": b_f, "b_u": b_u, "b_i":b_i, "b_c":b_c, "b_y":b_y, "a_prev": a_prev, "cell_prev": cell_prev}
    

    return(parameters)

In [51]:
parameters = backprop_lstm(X_trainmod, parameters, y_train ,1,0.1)

end of process


In [52]:
#initialize parameters: 
parameters = {} 

#dimension of every hidden neuron  
n = vectors[0].shape[0] #size of embeddings 
m = 10                  #every hidden neuron has the same dim as the input (1*m) m = 6
b = len(ranking)        #categories of the output 
z = 10                  #number of units for the cell state 
g = X_trainmod.shape[0] #number of samples 
#Initialize weights: 
parameters["forget"] = np.random.randn(m + n, z)
parameters["update"] = np.random.randn(m + n, z)
parameters["ihidden"] = np.random.randn(m + n, z)
parameters["candid"] = np.random.randn(m + n, z)
parameters["output_weights"] = np.random.randn(m,b)

#Initializing biases: 
parameters["b_f"] = np.random.randn(z)
parameters["b_u"] = np.random.randn(z)
parameters["b_i"] = np.random.randn(z) #?
parameters["b_c"] = np.random.randn(z)
parameters["b_y"] = np.random.randn(b)

#initial hidden and cell states 
parameters["a_prev"] = np.tile(np.random.randn(1,m), (g,1)).reshape(g,1,m) #this make a copy of a_prev so we initiate all hidden states for all samples at the same time 
parameters["cell_prev"] = np.tile(np.random.randn(1,z),(g,1)).reshape(g,1,z)

cache = {}


In [53]:
alpha = 0.1
iterations = 1
  
#first make a prediction based on the past? or maybe what it can receive is all the parameters after the lstm_forward step? coz it already 
#has it all? so what should we do? we take all the parameters as input. in this step you should already have run the lstm forward. 
#then we do what? take the lstm forward outputs and unwrap cache for gates outputs and parameters for the weights, biases, and the a_prev 
#and cell_prev. now in this case, you can run a reversed i iteration so that if i == last iteration, then we take the difference between the 
#prediction and the y_actual. then we move thrgouh the iterations back propagating this difference to update all parameters to reflect this change. 
    
n, T_x ,emb = X_trainmod.shape
p_y = 2                   #number of categories (negative vs positive ->2) 
    
    #define the y_actual: 
y_trainmod = np.zeros((n,p_y))
    
for i in range(n): 
        
    y_trainmod[i, :] = vec_output(y_train[i])

     




    #store the number of hidden neurons: 
n_a = w_y.shape[0]

    #start the iterations: 
for j in range(iterations):
        
        #make a prediction: 
    y_pred, caches, parameters = lstm_forward(X_trainmod, parameters)#this line can be problematic if the lstm_forward is previously run. 
    #unpack the parameters:
    w_f = parameters["forget"]
    w_u = parameters["update"]
    w_i = parameters["ihidden"]
    w_c = parameters["candid"]
    w_y = parameters["output_weights"]
    
    b_f = parameters["b_f"]
    b_u = parameters["b_u"]
    b_i = parameters["b_i"]
    b_c = parameters["b_c"]
    b_y = parameters["b_y"]
    a_state = parameters["a_prev"]
    cell_state = parameters["cell_prev"]
        
        #calculate the loss function: 
    cost = y_trainmod - y_pred
        
    dht = np.dot(cost, w_y.T) 
    dwy = np.dot(dht.T,cost)
    dby = np.sum(cost, axis = 0, keepdims = True)
    dc_next,dh_next = 0,0 
    dwf, dwu, dwihidden, dwcandid = 0,0,0,0
    dbcandid, dbf, dbu, dbihidden = 0,0,0,0
        
    for i in reversed(range(T_x)): 
            #load your cache: 
        ft, ut, candidt, ihiddent = caches[i] 
            #we take the 50th element of cell state (which is actually the last one) 
        dht = dht + dh_next 
        dct = dht * ihiddent * (1 - (np.tanh(cell_state[:,i+1,:]))**2) + dc_next 
        dcandidt = dct * ut * (1-candidt**2)

            #gate derivatives: 
        dihiddent = dht * np.tanh(cell_state[:,i+1, :]) * ihiddent * (1-ihiddent)
        dft = dct * cell_state[:,i, :] * ft * (1-ft)
        dut = dct * candidt * ut * (1-ut)
        
            #update the weights: 
        dwf += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dft) #change dims to vars. 
        dwu += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dut)
        dwihidden += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dihiddent)
        dwcandid += np.dot(np.hstack([X_trainmod[:,i,:].reshape(n,emb),a_state[:,i,:]]).T,dcandidt)

        dbcandid += np.sum(dcandidt, axis=0, keepdims=True) #add all the rows together. (1125 by 3 collapsed to 1 by 3) 
        dbf += np.sum(dft, axis = 0, keepdims = True) 
        dbu += np.sum(dut, axis=0, keepdims=True) 
            
        dbihidden += np.sum(dihiddent, axis=0, keepdims=True) 
        dh_next = np.dot(dft, w_f[-10:, :].T) + np.dot(dcandidt, w_c[-10:, :].T) + np.dot(dihiddent, w_i[-10:, :].T) + np.dot(dut, w_u[-10:, :].T)
        dc_next = dct * ft 

        #update the parameters: 
    w_f = w_f - alpha * dwf
    w_u = w_u -alpha * dwu
    w_i = w_i - alpha * dwihidden
    w_c = w_c - alpha * dwcandid 
    w_y = w_y -alpha * dwy 
        
    b_f = b_f - alpha * dbf 
    b_u = b_u - alpha * dbu 
    b_c = b_c - alpha * dbcandid
    b_i = b_i - alpha * dbihidden
    b_y = b_y - alpha * dby 

    #Re-initialize the variables a_prev and cell_prev for the next run: 
    a_prev = np.tile(np.random.randn(1,n_a), (n,1)).reshape(n,1,n_a) #this make a copy of a_prev so we initiate all hidden states for all samples at the same time 
    cell_prev = np.tile(np.random.randn(1,n_a),(n,1)).reshape(n,1,n_a)

        #Delete the content of the caches: 
    caches = []

        #Save the updated parameters: 
    parameters = {
        "forget": w_f, "update": w_u, "ihidden": w_i, "candid": w_c, "output_weights": w_y, 
        "b_f": b_f, "b_u": b_u, "b_i":b_i, "b_c":b_c, "b_y":b_y, 
        "a_prev": a_prev, "cell_prev": cell_prev}
    



NameError: name 'w_y' is not defined

In [ ]:
y_pred, caches, parameters = lstm_forward(X_trainmod, parameters)

In [ ]:
y_pred